# Validate Optimality of Morse Code

In [ ]:
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.functions.row_number

import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder.appName("Scala Morse Code Optimality Analysis").getOrCreate()
 
import spark.implicits._

def length(s: String): Int = {
    s.toList.foldLeft(0)((r, c) => r + (if (c == '.') 1 else 3))
}

val text = spark.sparkContext.textFile("/home/jovyan/Resources/sherlock-holmes.txt")
val lowr = text.flatMap(_.toUpperCase).filter(s => s >= 'A' && s <= 'Z').map(c => c.toString)

val pair = lowr.map(c => (c, 1))
val cnts = pair.reduceByKey(_ + _).toDF("letter", "count").orderBy("count").persist

cnts.rdd.collect.foreach(println(_))

val mrse = spark.sparkContext.textFile("/home/jovyan/Resources/morse-code.txt")
val code = mrse.map(s => s.split(" ")).map(s => (s(0), length(s(1)))).toDF("letter", "length").orderBy("length").persist
code.rdd.collect.foreach(println(_))

val desc = cnts.join(code, "letter").persist
desc.createOrReplaceTempView("desc")

val anom = spark.sql("select * from desc l join desc r on r.count > l.count and r.length > l.length")
anom.rdd.collect.foreach(println(_))
